!pip install dialogflow gensim==3.6.0 annoy tqdm stop_words pymorphy2 python-telegram-bot==13.3

!pip install pandarallel

<img src='pic_16_nlp_1.jpg' width=700>

<img src='pic_16_nlp_2.jpg' width=1000>

<img src='pic_16_nlp_3.jpg' width=1000>

In [1]:
pip freeze

absl-py==1.4.0
aiohttp==3.8.3
aiosignal==1.3.1
annoy==1.17.1
anyio @ file:///opt/concourse/worker/volumes/live/485b0f52-1188-482a-6285-65a36c8fa8a6/volume/anyio_1644481714856/work/dist
appnope @ file:///opt/concourse/worker/volumes/live/6ca6f098-d773-4461-5c91-a24a17435bda/volume/appnope_1606859448531/work
APScheduler==3.6.3
argon2-cffi @ file:///opt/conda/conda-bld/argon2-cffi_1645000214183/work
argon2-cffi-bindings @ file:///opt/concourse/worker/volumes/live/42cf1b28-e71f-45ed-47b2-50f828088636/volume/argon2-cffi-bindings_1644569709119/work
asttokens @ file:///opt/conda/conda-bld/asttokens_1646925590279/work
astunparse==1.6.3
async-timeout==4.0.2
attrs @ file:///private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_33k1uces4n/croot/attrs_1668696162258/work
backcall @ file:///home/ktietz/src/ci/backcall_1611930011877/work
beautifulsoup4 @ file:///private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_croot-cdiouih5/beautifulsoup4_1650462164803/work
bleach @ file:///opt/cond

Note: you may need to restart the kernel to use updated packages.


In [2]:
#.   RESTART KERNEL

In [3]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import pickle
    
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [4]:
import mmap
import re

def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

In [5]:
# Скачать файл Otvety.txt по ссылке  (1,7G)
# https://drive.google.com/file/d/1DQL9ybca4USImUDaxxHmkIZNmClKBtKG/view

In [6]:
# Преобразование файла аопросов-ответов в строчный вид
if not os.path.isfile('prepared_answers.txt'):

    question = None
    written = False

    with open("prepared_answers.txt", "w") as fout:  
        with open("Otvety.txt", "r") as fin:
            for line in tqdm_notebook(fin):
                if line.startswith("---"):
                    written = False
                    continue
                if not written and question is not None:
                    fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                    written = True
                    question = None
                    continue
                if not written:
                    question = line.strip()
                    continue

In [7]:
# Препроцессинг текста
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [8]:
# Функция для очистки текста из статьи https://habr.com/ru/articles/738176/ адаптированная под русский язык
def clean_text(input_text):

    # HTML-теги: первый шаг - удалить из входного текста все HTML-теги
    clean_text = re.sub('<[^<]+?>', '', input_text)

    # URL и ссылки: далее - удаляем из текста все URL и ссылки
    clean_text = re.sub(r'http\S+', '', clean_text)

    #Эмоджи и эмотиконы: используем собственную функцию для преобразования эмоджи в текст
    #Важно понимать эмоциональную окраску обрабатываемого текста
    clean_text = emojis_words(clean_text)

    # Приводим все входные данные к нижнему регистру
    clean_text = clean_text.lower()

    # Убираем все пробелы
    # Так как все данные теперь представлены словами - удалим пробелы
    clean_text = re.sub('\s+', ' ', clean_text)

    #Убираем специальные символы: избавляемся от всего, что не является "словами"
    clean_text = re.sub('[^а-яА-ЯёЁ0-9\s]', '', clean_text) #

    # Записываем числа прописью: 100 превращается в "сто" (для компьютера)# не работает
    temp = inflect.engine()
    words = []
    for word in clean_text.split():
        if word.isdigit():
            words.append(num2words(int(word), lang='ru'))
        else:
            words.append(word)
    clean_text = ' '.join(words)

        # Стоп-слова: удаление стоп-слов - это стандартная практика очистки текстов
    stop_words = set(stopwords.words('russian'))
    tokens = word_tokenize(clean_text)
    tokens = [token for token in tokens if token not in stop_words]
    clean_text = ' '.join(tokens)

    # Знаки препинания: далее - удаляем из текста все знаки препинания
    clean_text = re.sub(r'[^\w\s]', '', clean_text)

    # И наконец - возвращаем очищенный текст
    return clean_text

# Функция для преобразования эмоджи в слова
def emojis_words(text):

    # Модуль emoji: преобразование эмоджи в их словесные описания
    clean_text = emoji.demojize(text, delimiters=(" ", " "))

    # Редактирование текста путём замены ":" и" _", а так же - путём добавления пробела между отдельными словами
    clean_text = clean_text.replace(":", "").replace("_", " ")

    return clean_text

def preproc_text(text):
    res = str(text).strip()
    res = re.sub(r"\s+", " ", res)
    return res

def build_data(data_q, data_ans):
    data = []
    for idx, texts in enumerate(data_q):
        question = preproc_text(texts)
        answer = preproc_text(data_ans.iloc[idx])
        res = '\nx:' + question + '\ny:' + answer
        data.append(res)
    return data

In [9]:
# Обработка текста

sentences = []
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

file_path_from = 'prepared_answers.txt'
file_path_to = 'Otvety2.txt'

if not os.path.isfile(file_path_to):
    
    N = get_num_lines(file_path_from)
    with open(file_path_to, mode = 'w') as fileto:
        with open(file_path_from) as filefrom:
            for k in tqdm(range(N)):
                line = filefrom.readline()
                if line == '': break
                spls = preprocess_txt(line)
                sentences.append(spls)
                c += 1
                if c > 500000000:
                    break
                fileto.write(' '.join(spls)+'\n')
    filefrom.close()
    fileto.close()

In [10]:
# Загрузить результат

sentences = []

file_path_from = 'Otvety2.txt'
if os.path.isfile(file_path_from):  
    N = get_num_lines(file_path_from) 
    with open(file_path_to, mode = 'r') as filefrom:
        for k in tqdm(range(N)):
            line = filefrom.readline()
            if line == '': break
            sentences.append(line.split())
    filefrom.close()

  0%|          | 0/7016256 [00:00<?, ?it/s]

In [11]:
vec = []
_ = [vec.extend(x)  for x in sentences[:100]]
vec = list(set(vec))
vec.sort()
vec[20:30]

['brconvert',
 'brps',
 'brв',
 'brвампир',
 'brвитание',
 'brвсе',
 'brгде',
 'brдикобраз',
 'brести',
 'brзабористый']

In [12]:
#    <br>

In [13]:
#df['text'] = value_counts(normalize=True)

In [14]:
sentences[:10]

[[],
 [],
 ['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести'],
 ['хомячок'],
 ['мужик', 'йопарить', 'собачка', '50', 'кошка'],
 ['общение'],
 ['паучок'],
 ['пол', 'памытьbr', 'таг', 'тип', 'каво'],
 ['вообще', 'пообщаться'],
 ['советовать', 'сися', 'завести']]

In [15]:
#sentences = sentences[:50_000]

In [16]:
# Обучим модель FastText

file_path_from = 'ft_model'
if not os.path.isfile(file_path_from):  
    
    sentences = [i for i in tqdm(sentences) if len(i) > 2]
    modelFT = FastText(sentences=sentences, size=100, min_count=1, window=5)
    modelFT.save("ft_model")

In [17]:
# Загрузить модель

modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

In [18]:
#Размерность вектора модели. Если установить 100 — каждое слово в корпусе будет представлено в виде 100-мерного вектора, и т.п.
#Наименьшее допустимое количество символов в слове, для которого будет создаваться векторное представление; так можно убрать частотные, но не очень значимые слова типа союзов и предлогов.
#Размер окна. Этот параметр задает, сколько соседних слов считается частью контекста. Если выставить 40, то алгоритм возьмет 40 слов спереди от слова и 40 слов сзади от слова;

In [20]:
list(set(get_stop_words("ru")))[:20]

['ваша',
 'девятнадцать',
 'действительно',
 'сама',
 'семнадцатый',
 'нельзя',
 'которая',
 'этом',
 'это',
 'лучше',
 'пора',
 'около',
 'тот',
 'лет',
 'тому',
 'даром',
 'шесть',
 'том',
 'всё',
 'многочисленные']

In [21]:
# Создаем Индексы для вопросов-ответов

file_path_from = 'speaker.ann'
if not os.path.isfile(file_path_from):  
    morpher = MorphAnalyzer()
    sw = set(get_stop_words("ru"))
    exclude = set(string.punctuation)
    modelFT = FastText.load("ft_model")
    ft_index = annoy.AnnoyIndex(100 ,'angular')

    index_map = {}
    counter = 0
    with open("Otvety2.txt", "r") as f:
        for line in tqdm(f):
            n_ft = 0
            spls = line.split("\t")
            index_map[counter] = spls[1]
            question = preprocess_txt(spls[0])
            vector_ft = np.zeros(100)
            for word in question:
                if word in modelFT.wv:
                    vector_ft += modelFT.wv[word]
                    n_ft += 1
            if n_ft > 0:
                vector_ft = vector_ft / n_ft
            ft_index.add_item(counter, vector_ft)

            counter += 1

            if counter > 50_000:
                break

    ft_index.build(10)
    ft_index.save('speaker.ann')
    
    with open("index_speaker.pkl", "wb") as f:
        pickle.dump(index_map, f)

In [22]:
#  Загрузим индексы
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann')
index_map = pd.read_pickle("index_speaker.pkl")

In [23]:
np.random.permutation(100)

array([69, 35, 71, 17, 78, 38, 46,  8, 88, 90, 85, 30, 29, 18, 32, 27, 25,
       87, 62,  6, 44, 39, 56, 12, 53, 52, 10, 77, 95, 22, 94, 83, 67, 72,
       48, 33, 55, 74, 36, 79, 68, 63, 45,  7, 51, 75, 50, 66,  4, 23, 64,
       24, 41, 92, 76, 81, 14, 59, 97, 40, 60, 84, 86, 93, 11, 98, 80, 34,
       58, 96, 13,  1, 19, 42, 99,  3, 54, 91,  0, 26, 70, 82, 21, 20, 57,
       16, 37, 89, 43, 31, 65, 73, 49, 28,  5, 47, 15,  9, 61,  2])

In [24]:
#  Пример получения индексов
a = ft_index.get_nns_by_vector(np.random.permutation(100), 5, include_distances=True)
a

([5468, 14291, 29194, 32866, 31658],
 [1.2329422235488892,
  1.2337841987609863,
  1.2337841987609863,
  1.2337841987609863,
  1.234498381614685])

In [25]:
[index_map[x] for x in a[0]]

['на выходных закончились =). \n',
 'меню зависло. \n',
 '2. Бельгийский аппарат для измерения силы детонации порохового заряда, вариант порохового тестера XVI века.<br>Самый ранний аппарат для проверки силы ружейного пороха был изобретен Берном в 1578 году. Это был небольшой цилиндр с прочно прилегающей крышкой на петле. Угол, на который поднималась крышка при взрыве внутри механизма, указывал силу пороха.<br><br>Непонятно. Зачем стоит скобка на нижней дуге механизма? Ограничитель засыпки пороха,<br>который снимается, затем, при тестировании?. \n',
 'угар. \n',
 'В любом продовольственном магазине Они могут быть под разными названиями<br>. \n']

In [ ]:
# https://gbcdn.mrgcdn.ru/uploads/asset/5209459/attachment/1b2f5aa57ff77e7c2d2ee26ceb09eb9e.csv

In [26]:
# Создадим модель продуктовых данных

shop_data = pd.read_csv("ProductsDataset.csv")
#shop_data = shop_data.iloc[:5000, :]

shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].progress_apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

  0%|          | 0/35548 [00:00<?, ?it/s]

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [27]:
# Подготовка для создания модели для определения домена данных

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [28]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
# Вопрос-ответный домен
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in tqdm(idxs)]
# Продуктовый домен
positive_texts = [" ".join(val) for val in tqdm(shop_data['text'].values)]

  0%|          | 0/25362 [00:00<?, ?it/s]

  0%|          | 0/35548 [00:00<?, ?it/s]

In [29]:
# ВО = 0, Прод = 1

dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, 
                                                    stratify=labels, random_state=13)

In [31]:
# Модель

x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

In [32]:
# Качество

from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9760302085043506

In [33]:
# Добавим IDF взвешивание (для каждого слова найдем IDF вес)

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

In [35]:
np.mean(tfidf_vect.idf_)

10.771440512796

In [36]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}

In [37]:
len(idfs.keys())

150383

In [38]:
list(idfs.keys())[:5]

['новый', 'куртка', 'adidas', 'демисезонный', 'фирма']

In [39]:
list(idfs.values())[:5]

[9.309122964599505,
 6.981845259015088,
 10.184591701953405,
 11.100882433827561,
 11.100882433827561]

In [40]:
# Создаем Индексы для продуктовых данных

file_path_from = 'shop.ann'
if not os.path.isfile(file_path_from):  
    
    
    ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

    midf = np.mean(tfidf_vect.idf_)

    index_map_shop = {}
    counter = 0

    for i in tqdm(range(len(shop_data))):
        n_ft = 0
        index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
        vector_ft = np.zeros(100)
        for word in shop_data.loc[i, "text"]:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word] * idfs.get(word, midf)
                n_ft += idfs.get(word, midf)
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index_shop.add_item(counter, vector_ft)
        counter += 1

    ft_index_shop.build(10)
    ft_index_shop.save('shop.ann')

    file_path_from = 'index_shop.pkl'
    if not os.path.isfile(file_path_from):  
    
        with open("index_shop.pkl", "wb") as f:
            pickle.dump(index_map_shop, f)


In [41]:
# Загрузим индексы

midf = np.mean(tfidf_vect.idf_)

ft_index_shop = annoy.AnnoyIndex(100, 'angular')
ft_index_shop.load('shop.ann') 

index_map_shop = pd.read_pickle("index_shop.pkl")

In [42]:
# Основная функция преобразования текста в вектор х100

def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [43]:
# Пример получения индекса

ft_index_shop.get_nns_by_vector(np.ones(100)*20, 5, include_distances=True)

([4460, 17938, 4897, 6088, 468],
 [1.20292329788208,
  1.2040966749191284,
  1.2047914266586304,
  1.205664038658142,
  1.2065114974975586])

In [44]:
# Создание своего бота в телеграмм
# @botfather
# /start
# /newbot - create a new bot
# name1
# name1_BOT
#. ->. API

In [47]:
# заменить на свой токен
updater = Updater("6523778836:AAH4__N3uT8Jk5DEX8yBpnD8rs0BAAInzsA", use_context=True) # Токен API к Telegram
dispatcher = updater.dispatcher

def startCommand(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text='Привет, давай пообщаемся?')

def textMessage(update, context):
    
    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)
    
    # ПРОД
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            context.bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return
    
    # QA
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    
    # 
    if distances[0] > 100.5:
        print(distances[0])
        context.bot.send_message(chat_id=update.message.chat_id, text="Моя твоя не понимать")
        return
    
    # Вопрос-Ответ
    context.bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])
        
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

/var/folders/4s/fd42fwqn6g746zjn0s08wmxm0000gn/T/ipykernel_1423/2641577093.py:10: RuntimeWarning: invalid value encountered in divide
  return vector_ft / n_ft
No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/Users/lmv/opt/anaconda3/envs/nlp/lib/python3.9/site-packages/telegram/ext/dispatcher.py", line 442, in process_update
    handler.handle_update(update, self, check, context)
  File "/Users/lmv/opt/anaconda3/envs/nlp/lib/python3.9/site-packages/telegram/ext/handler.py", line 160, in handle_update
    return self.callback(update, context)
  File "/var/folders/4s/fd42fwqn6g746zjn0s08wmxm0000gn/T/ipykernel_1423/1419090691.py", line 33, in textMessage
    context.bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])
  File "/Users/lmv/opt/anaconda3/envs/nlp/lib/python3.9/site-packages/telegram/bot.py", line 126, in decorator
    result = func(*args, **kwargs)
  File "/Users/lmv/opt/anaconda3/envs/nlp/lib/python

In [ ]:
#  https://github.com/Koziev/NLP_Datasets